<a href="https://colab.research.google.com/github/Vonewman/Audio-course-hf/blob/main/Mod%C3%A8les_pr%C3%A9_entra%C3%AEn%C3%A9s_pour_la_reconnaissance_automatique_de_la_parole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modèles pré-entraînés pour la reconnaissance automatique de la parole

In [ ]:
!pip install -q transformers[sentencepiece] datasets accelerate

## Sonder les modèles CTC

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "hf-internal-testing/librispeech_asr_dummy", "clean", split="validation"
)
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 73
})

In [ ]:
from IPython.display import Audio

sample = dataset[2]

print(sample["text"])
Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

HE TELLS US THAT AT THIS FESTIVE SEASON OF THE YEAR WITH CHRISTMAS AND ROAST BEEF LOOMING BEFORE US SIMILES DRAWN FROM EATING AND ITS RESULTS OCCUR MOST READILY TO THE MIND


In [ ]:
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-100h")

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-100h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.mask_time_emb_vector']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-100h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametriz

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

In [ ]:
pipe(sample["audio"].copy())
{
    "text": "HE TELLS US THAT AT THIS FESTIVE SEASON OF THE YEAR WITH CHRISTMAUS AND ROSE BEEF LOOMING BEFORE US SIMALYIS DRAWN FROM EATING AND ITS RESULTS OCCUR MOST READILY TO THE MIND"
}

{'text': 'HE TELLS US THAT AT THIS FESTIVE SEASON OF THE YEAR WITH CHRISTMAUS AND ROSE BEEF LOOMING BEFORE US SIMALYIS DRAWN FROM EATING AND ITS RESULTS OCCUR MOST READILY TO THE MIND'}

In [ ]:
import torch
from transformers import pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline(
    "automatic-speech-recognition", model="openai/whisper-base", device=device
)

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
pipe(sample["audio"], max_new_tokens=256)

{'text': ' He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similarly is drawn from eating and its results occur most readily to the mind.'}

In [ ]:
dataset = load_dataset(
    "facebook/multilingual_librispeech", "french", split="validation", streaming=True
)
sample = next(iter(dataset))

In [ ]:
print(sample["text"])
Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

le salon était de la plus haute magnificence doré comme la galerie de diane aux tuileries avec des tableaux à l'huile au lambris il y avait des taches claires dans ces tableaux julien apprit plus tard que les sujets avaient semblé peu décents à la maîtresse du logis qui avait fait corriger les tableaux


In [ ]:
pipe(sample["audio"].copy(), max_new_tokens=256, generate_kwargs={"task": "transcribe"})

{'text': " Le salon était de la plus haute magnificence, doré comme la galerie de Dianne aux Tulleries, avec tes tableaux à l'huile au l'embry. Il y avait des tâches claires dans ces tableaux. J'y viens à prix plus tard que les sujets avaient semblé peu-décent à la maîtresse du logis, qui avait fait corriger les tableaux."}

In [ ]:
pipe(sample["audio"], max_new_tokens=256, generate_kwargs={"task": "translate"})

{'text': ' The salon was of the highest significance, golden as the gallery of Diana in Tullerie, with the tables at the back. There were clear dates in this table. I just took later that the subjects had looked more decent at the metallurgy, which had made the tables correct.'}

## Longue transcription et horodatage

In [ ]:
import numpy as np

target_length_in_m = 5

# convertir les minutes en secondes (* 60) en nombre d'échantillons (* taux d'échantillonnage)
sampling_rate = pipe.feature_extractor.sampling_rate
target_length_in_samples = target_length_in_m * 60 * sampling_rate

# itérer sur notre jeu de données en streaming, en concaténant les échantillons jusqu'à ce que nous atteignions notre cible
long_audio = []
for sample in dataset:
    long_audio.extend(sample["audio"]["array"])
    if len(long_audio) > target_length_in_samples:
        break

long_audio = np.asarray(long_audio)

# Résultat
seconds = len(long_audio) / 16000
minutes, seconds = divmod(seconds, 60)
print(f"Length of audio sample is {minutes} minutes {seconds:.2f} seconds")

Length of audio sample is 5.0 minutes 6.98 seconds


In [ ]:
pipe(
    long_audio,
    max_new_tokens=256,
    generate_kwargs={"task": "transcribe"},
    chunk_length_s=30,
    batch_size=8,
)

{'text': " Le salon était de la plus haute magnificence, doré comme la galerie de Dianne aux Tulleries, avec tes tableaux à l'huile au l'embris. Il y avait des tâches claires dans ces tableaux. J'y viens à prix plus tard que les sujets avaient semblé peu décent à la maîtresse du logis, qui avait fait corriger les tableaux. Elle avait plus d'intérêt quand Madame de Fervac venait chez la marqueuse. Alors, il pouvait entrevoir les yeux de Matilde, sous une elle du chapeau de la Maréchale, et il était éloquant. Ces phrases pittoresques et sentimentales commencent à prendre une tournure plus frappante à la fois et plus élégante. Vient la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur de la chaleur d

In [ ]:
pipe(
    long_audio,
    max_new_tokens=256,
    generate_kwargs={"task": "transcribe"},
    chunk_length_s=30,
    batch_size=8,
    return_timestamps=True,
)["chunks"]

[{'timestamp': (0.0, 6.64),
  'text': ' Le salon était de la plus haute magnificence, doré comme la galerie de Dianne aux Tulleries,'},
 {'timestamp': (6.64, 11.76),
  'text': " avec tes tableaux à l'huile au l'embry. Il y avait des tâches claires dans ces tableaux."},
 {'timestamp': (11.76, 16.16),
  'text': " J'y viens à prix plus tard que les sujets avaient semblé peu-décent à la maîtresse"},
 {'timestamp': (16.16, 21.04),
  'text': " tue logie, qui avait fait corriger les tableaux. Elle avait plus d'intérêt quand Madame"},
 {'timestamp': (21.04, 25.04),
  'text': ' de Fervac venait chez la Marquise, alors il pouvait entrevoir'},
 {'timestamp': (25.04, 30.42),
  'text': ' les yeux de Matilde, sous une elle du chapeau de la Maréchale, et il était éloquant.'},
 {'timestamp': (30.42, 36.68),
  'text': ' Ces phrases pittoresques et sentimentales commencent à prendre une tournure plus frappante à la'},
 {'timestamp': (36.68, 38.48), 'text': ' fois et plus élégante.'},
 {'timestamp': (38.